In [1]:
from transformers import AutoModel, AutoTokenizer
import torch
import json
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
model = AutoModel.from_pretrained(
    'mesolitica/llama2-embedding-1b-8k-contrastive',
    use_flash_attention_2 = True, 
    torch_dtype = torch.bfloat16,
    trust_remote_code = True,
)

[2023-12-04 04:07:18,202] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [3]:
_ = model.cuda()

In [4]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-1b-8k-contrastive')

In [5]:
contexts, questions = [], []

with open('/home/ubuntu/crossref-doi/ultrachat-crossref-melayu-malay.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l[1]['content_ms']:
            c = l[0]['content'].strip()
            q = l[1]['content_ms'].strip()
            if not len(c) or not len(q):
                continue
            contexts.append(c)
            questions.append(q)
            
len(contexts), len(questions)

(9673, 9673)

In [6]:
contexts_v = []
for i in tqdm(range(len(contexts))):
    input_ids = tokenizer([contexts[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    contexts_v.append(v)

100%|██████████| 9673/9673 [07:58<00:00, 20.22it/s]


In [7]:
questions_v = []
for i in tqdm(range(len(contexts_v))):
    input_ids = tokenizer([questions[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    questions_v.append(v)

100%|██████████| 9673/9673 [00:54<00:00, 176.23it/s]


In [8]:
contexts_v_np = np.array(contexts_v)[:,0]
questions_v_np = np.array(questions_v)[:,0]

In [9]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}

for i in tqdm(range(len(questions_v_np))):
    argsort = np.argsort(cosine_similarity(questions_v_np[i].reshape(1, -1), contexts_v_np)[0])[::-1]
    for k in tops.keys():
        if i in argsort[:k]:
            tops[k] += 1

100%|██████████| 9673/9673 [04:20<00:00, 37.14it/s]


In [10]:
for k, v in tops.items():
    print(k, v / len(questions_v_np))

1 0.16489196733174816
3 0.3281298459629898
5 0.42623798201178537
10 0.5487439263930528
